## This is for capstone project to complete Coursera's IBM Data Science Certificate

### Author: Tansaya Kunaratskul


### Table of Content
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
    * [K-Means Clustering](#kmeans)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

<a id='introduction'></a>
## Introduction: Business Problem

<br>This case requires to develop a customer segmentation to define marketing strategy. The
sample Dataset summarizes the usage behavior of about 9000 active credit card holders during the last 6 months. The file is at a customer level with 18 behavioral variables.

We want to cluser the credit card customers into segments using the attributes provided. Given the cluster, we hope to see if there are certain customer behaviors we can extract from each cluster and thus, the company will be able to offer personalized financial products for each of them.

<a id='data'></a>
## Data
<br>There are total of 8950 active credit cards in our data sets in the last 6 months. For the data, we have transactin behaviors for each one.

Following is the Data Dictionary for Credit Card dataset :-
1. CUSTID : Identification of Credit Card holder (Categorical)
2. BALANCE : Balance amount left in their account to make purchases (
3. BALANCEFREQUENCY : How frequently the Balance is updated, score between 0 and 1 (1 = frequently updated, 0 = not frequently updated)
4. PURCHASES : Amount of purchases made from account
5. ONEOFFPURCHASES : Maximum purchase amount done in one-go
6. INSTALLMENTSPURCHASES : Amount of purchase done in installment
7. CASHADVANCE : Cash in advance given by the user
8. PURCHASESFREQUENCY : How frequently the Purchases are being made, score between 0 and 1 (1 = frequently purchased, 0 = not frequently purchased)
9. ONEOFFPURCHASESFREQUENCY : How frequently Purchases are happening in one-go (1 = frequently purchased, 0 = not frequently purchased)
10. PURCHASESINSTALLMENTSFREQUENCY : How frequently purchases in installments are being done (1 = frequently done, 0 = not frequently done)
11.  CASHADVANCEFREQUENCY : How frequently the cash in advance being paid
12.CASHADVANCETRX : Number of Transactions made with "Cash in Advanced"
13. PURCHASESTRX : Numbe of purchase transactions made
14. CREDITLIMIT : Limit of Credit Card for user
15. PAYMENTS : Amount of Payment done by user
16. MINIMUM_PAYMENTS : Minimum amount of payments made by user
17. PRCFULLPAYMENT : Percent of full payment paid by user
18. TENURE : Tenure of credit card service for user

In [2]:
# import necessary libraries
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from scipy import ndimage 
from scipy.cluster import hierarchy 
from scipy.spatial import distance_matrix 
from sklearn.cluster import KMeans 
from sklearn import manifold, datasets 
from sklearn.cluster import AgglomerativeClustering 
from sklearn.datasets.samples_generator import make_blobs 
from sklearn import preprocessing
%matplotlib inline

In [5]:
# import data
df = pd.read_csv (r'/Users/benz/Downloads/CC GENERAL.csv')
df.head(5)

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12


In [8]:
# look at the data type and the statistical summary for each variable
print(df.dtypes)
print(df.describe())

CUST_ID                              object
BALANCE                             float64
BALANCE_FREQUENCY                   float64
PURCHASES                           float64
ONEOFF_PURCHASES                    float64
INSTALLMENTS_PURCHASES              float64
CASH_ADVANCE                        float64
PURCHASES_FREQUENCY                 float64
ONEOFF_PURCHASES_FREQUENCY          float64
PURCHASES_INSTALLMENTS_FREQUENCY    float64
CASH_ADVANCE_FREQUENCY              float64
CASH_ADVANCE_TRX                      int64
PURCHASES_TRX                         int64
CREDIT_LIMIT                        float64
PAYMENTS                            float64
MINIMUM_PAYMENTS                    float64
PRC_FULL_PAYMENT                    float64
TENURE                                int64
dtype: object
            BALANCE  BALANCE_FREQUENCY     PURCHASES  ONEOFF_PURCHASES  \
count   8950.000000        8950.000000   8950.000000       8950.000000   
mean    1564.474828           0.877271   1003.

In [13]:
# Data Cleaning
# see if there is any NaN
missing_data = df.isnull()
missing_data.head(5)

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [14]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")   

CUST_ID
False    8950
Name: CUST_ID, dtype: int64

BALANCE
False    8950
Name: BALANCE, dtype: int64

BALANCE_FREQUENCY
False    8950
Name: BALANCE_FREQUENCY, dtype: int64

PURCHASES
False    8950
Name: PURCHASES, dtype: int64

ONEOFF_PURCHASES
False    8950
Name: ONEOFF_PURCHASES, dtype: int64

INSTALLMENTS_PURCHASES
False    8950
Name: INSTALLMENTS_PURCHASES, dtype: int64

CASH_ADVANCE
False    8950
Name: CASH_ADVANCE, dtype: int64

PURCHASES_FREQUENCY
False    8950
Name: PURCHASES_FREQUENCY, dtype: int64

ONEOFF_PURCHASES_FREQUENCY
False    8950
Name: ONEOFF_PURCHASES_FREQUENCY, dtype: int64

PURCHASES_INSTALLMENTS_FREQUENCY
False    8950
Name: PURCHASES_INSTALLMENTS_FREQUENCY, dtype: int64

CASH_ADVANCE_FREQUENCY
False    8950
Name: CASH_ADVANCE_FREQUENCY, dtype: int64

CASH_ADVANCE_TRX
False    8950
Name: CASH_ADVANCE_TRX, dtype: int64

PURCHASES_TRX
False    8950
Name: PURCHASES_TRX, dtype: int64

CREDIT_LIMIT
False    8949
True        1
Name: CREDIT_LIMIT, dtype: int64

PAYMENTS

We see that for te variable 'MINIMUM_PAYMENTS' and 'CREDIT_LIMIT', there are missing values. 

<br> To deal with the missing values, we will replace by <b>the average number</b>

In [6]:
#Replace average for 'MINIMUM_PAYMENTS' 
avg_min_payment = df["MINIMUM_PAYMENTS"].astype("float").mean(axis=0)
print("Average of MINIMUM_PAYMENTS:", avg_min_payment)

Average of MINIMUM_PAYMENTS: 864.2065423050814


In [7]:
# Replace NaN with the average
df["MINIMUM_PAYMENTS"].replace(np.nan, avg_min_payment, inplace=True)

In [8]:
# Do the same with 'CREDIT_LIMIT'
avg_credit_limit = df["CREDIT_LIMIT"].astype("float").mean(axis=0)
print("Average of CREDIT_LIMIT:", avg_credit_limit)

Average of CREDIT_LIMIT: 4494.449450364621


In [9]:
# Replace NaN with the average
df["CREDIT_LIMIT"].replace(np.nan, avg_credit_limit, inplace=True)

<a id='methodology'></a>
## Methodology 
<br> Given 18 credit card behavioral factors, we determine to cluser them in order to see if there are certain patterns that emerge from each cluser. For this project, we will use 3 clustering methods that we learned:
<br>1) K-Means Clustering
<br>2) Hierachical Clustering
<br>3) Density-based Clustering

<a id='kmeans'></a>

### 1) K-Means Clustering 
We start by normalizing over the standard deviation by using <b>StandardScaler()</b> to normalize our dataset.

In [10]:
from sklearn.preprocessing import StandardScaler
X = df.values[:,1:]
Clus_dataSet = StandardScaler().fit_transform(X)
Clus_dataSet

/Users/benz/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[-0.73198937, -0.24943448, -0.42489974, ..., -0.31096755,
        -0.52555097,  0.36067954],
       [ 0.78696085,  0.13432467, -0.46955188, ...,  0.08931021,
         0.2342269 ,  0.36067954],
       [ 0.44713513,  0.51808382, -0.10766823, ..., -0.10166318,
        -0.52555097,  0.36067954],
       ...,
       [-0.7403981 , -0.18547673, -0.40196519, ..., -0.33546549,
         0.32919999, -4.12276757],
       [-0.74517423, -0.18547673, -0.46955188, ..., -0.34690648,
         0.32919999, -4.12276757],
       [-0.57257511, -0.88903307,  0.04214581, ..., -0.33294642,
        -0.52555097, -4.12276757]])

#### Modeling
Lets apply k-means on our dataset, and take look at cluster labels.

In [ ]:
# Set number of cluster to 4
clusterNum = 4
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 12)
k_means.fit(X)
labels = k_means.labels_
print(labels)